In [ ]:
# ===============================
# Voice2Music – Clean Input to MIDI
# ===============================
import os
import sounddevice as sd
import soundfile as sf
import librosa
import pretty_midi
import numpy as np

# -------------------------------
# Config: folders & recording
# -------------------------------
DATA_FOLDER = r'D:\shree\Miniproject\voicetomusic\data'
OUTPUT_FOLDER = r'D:\shree\Miniproject\voicetomusic\output'
os.makedirs(DATA_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# -------------------------------
# Function: Record audio
# -------------------------------
def record_audio(filename='voice_sample.wav', duration=15, fs=44100):
    """Records audio from microphone and saves as WAV"""
    print(f"Recording for {duration} seconds...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    
    audio_path = os.path.join(DATA_FOLDER, filename)
    sf.write(audio_path, recording, fs)
    
    print(f"Audio saved at {audio_path}")
    return audio_path

# -------------------------------
# Function: Convert audio to MIDI
# -------------------------------
def audio_to_midi(audio_path):
    """Convert audio to MIDI and save in output folder"""
    y, sr = librosa.load(audio_path, sr=None)
    f0, voiced_flag, voiced_probs = librosa.pyin(
        y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7')
    )
    midi_notes = librosa.hz_to_midi(f0)

    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)  # Acoustic Grand Piano
    frame_duration = librosa.get_duration(y=y, sr=sr) / len(midi_notes)
    
    for i, note in enumerate(midi_notes):
        if np.isnan(note):
            continue
        pm_note = pretty_midi.Note(velocity=100, pitch=int(note),
                                   start=i*frame_duration, end=(i+1)*frame_duration)
        instrument.notes.append(pm_note)

    pm.instruments.append(instrument)
    
    # Correct output filename
    output_name = os.path.basename(audio_path).replace('.wav', '.mid')
    output_path = os.path.join(OUTPUT_FOLDER, output_name)
    pm.write(output_path)
    
    print(f"MIDI saved at {output_path}")
    return output_path

# -------------------------------
# Usage
# -------------------------------
# 1️⃣ Record new audio
audio_path = record_audio(filename='sample1.wav', duration=15)

# 2️⃣ Convert to MIDI
midi_path = audio_to_midi(audio_path)


Recording for 15 seconds...
Audio saved at D:\shree\Miniproject\voicetomusic\data\sample1.wav
MIDI saved at D:\shree\Miniproject\voicetomusic\output\sample1.mid
